In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
import torch
from sklearn.metrics import accuracy_score
from torch.cuda.amp import autocast

In [37]:
print(torch.cuda.is_available())


True


In [3]:
# Проверяем доступность GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
# Загрузка данных из CSV-файла
df = pd.read_csv('your_dataset.csv')  # Замените 'your_dataset.csv' на путь к вашему файлу CSV

# Разделение данных на обучающий и тестовый наборы
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Подготовка текстов и меток
train_texts, train_labels = train_df['news_headline'].tolist(), train_df['category_num'].tolist()
test_texts, test_labels = test_df['news_headline'].tolist(), test_df['category_num'].tolist()

In [5]:
# Инициализация токенизатора и преобразование текста в токены
tokenizer = AutoTokenizer.from_pretrained('distilroberta-base')
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')

# Преобразование меток в тензоры
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)

# Создание DataLoader для обучения и тестирования
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

# Используйте DataLoader для автоматического перемещения данных на GPU
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, pin_memory=True)

# Инициализация модели RoBERTa для классификации на GPU
num_classes = 2  # Замените 2 на количество классов в вашем наборе данных

# Загрузка оригинальной модели RoBERTa
original_model = AutoModelForSequenceClassification.from_pretrained('distilroberta-base')

# Создание новой (менее объемной) модели с использованием distill
distil_model = AutoModelForSequenceClassification.from_pretrained('distilroberta-base')
distil_model.to(device)

# Инициализация оптимизатора
optimizer = AdamW(distil_model.parameters(), lr=5e-5)

c:\Users\vvlrff\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vvlrff\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\vvlrff\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, o

In [10]:
# Обучение модели на GPU
num_epochs = 1

for epoch in range(num_epochs):
    distil_model.train()
    total_loss = 0.0
    correct_predictions = 0

    for batch_num, batch in enumerate(train_loader, 1):
        inputs = {'input_ids': batch[0].to(device), 'attention_mask': batch[1].to(device), 'labels': batch[2].to(device)}
        optimizer.zero_grad()

        with autocast():  # Используем mixed-precision training
            outputs = distil_model(**inputs)
            loss = outputs.loss

        loss.backward()
        optimizer.step()

        # Считаем общую потерю
        total_loss += loss.item()

        # Считаем количество правильных предсказаний
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)
        correct_predictions += torch.sum(predictions == batch[2].to(device)).item()  # Перемещаем метки на устройство GPU


    # Выводим лог в конце каждой эпохи
    avg_loss = total_loss / len(train_loader)
    accuracy = correct_predictions / (len(train_loader) * train_loader.batch_size)
    print(f'Epoch {epoch + 1}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}')

    # Проверка точности на тестовом наборе
    distil_model.eval()
    all_preds = []
    with torch.no_grad():
        for test_batch in test_loader:
            test_inputs = {'input_ids': test_batch[0].to(device), 'attention_mask': test_batch[1].to(device)}
            test_outputs = distil_model(**test_inputs)
            test_logits = test_outputs.logits
            test_preds = torch.argmax(test_logits, dim=1)
            all_preds.extend(test_preds.cpu().numpy())

    # Вычисление и вывод точности на тестовом наборе
    test_accuracy = accuracy_score(test_labels.cpu().numpy(), all_preds)
    print(f'Test Accuracy: {test_accuracy:.4f}')

Epoch 1, Loss: 0.0906, Accuracy: 0.9702
Test Accuracy: 0.9852


In [11]:
# Сохранение модели
distil_model.save_pretrained('distil_roberta_model')